In [1]:
pip -q install peft transformers langchain sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 67.2 MB/s eta 0:00:00


In [2]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

In [4]:
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
device_map = {"": "cpu"}

#model = AutoPeftModelForCausalLM.from_pretrained(adapt_model_name, device_map=device_map, torch_dtype=torch.bfloat16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Load model from HuggingFace Hub
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [6]:
def get_embedding(sentence):

    #Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

In [7]:
answers = []
emb_database = torch.empty((0, 384), dtype=torch.float32)

In [16]:
questions_hse = [
    "Какие факультеты и программы предлагает НИУ ВШЭ?",
    "Каковы требования для поступления в НИУ ВШЭ?",
    "Какие документы необходимо подать для поступления в НИУ ВШЭ?",
    "Есть ли вступительные экзамены или тесты для поступления в НИУ ВШЭ, и как к ним подготовиться?",
    "Предоставляются ли стипендии или гранты для студентов НИУ ВШЭ?",
    "Какие карьерные возможности открываются для выпускников НИУ ВШЭ?",
    "Есть ли у НИУ ВШЭ международные программы обмена студентами?",
    "Какие возможности для стажировок и практики доступны студентам НИУ ВШЭ?",
    "Какова средняя группа студентов на лекциях и семинарах в НИУ ВШЭ?",
    "Есть ли общежития для иногородних студентов в НИУ ВШЭ?",
    "Как организована внеучебная деятельность и студенческая жизнь в НИУ ВШЭ?",
    "Есть ли в НИУ ВШЭ спортивные секции и клубы по интересам?",
    "Какие возможности для научной работы и исследований предоставляются студентам НИУ ВШЭ?",
    "Какова процедура подачи заявки для иностранных студентов в НИУ ВШЭ?",
    "Какой уровень владения языком требуется для поступления в НИУ ВШЭ?",
    "Какие услуги предоставляет НИУ ВШЭ для поддержки студентов (например, психологическая помощь, карьерные консультации)?",
    "Каковы сроки подачи документов и зачисления в НИУ ВШЭ?",
    "Есть ли у НИУ ВШЭ партнерские отношения с крупными компаниями или предприятиями?",
    "Какова процедура апелляции в случае несогласия с результатами вступительных экзаменов в НИУ ВШЭ?"
]

In [8]:
data_llama = [
    "НИУ ВШЭ предлагает следующие факультеты и программы: Экономика, менеджмент; Компьютерные науки, управление; Юриспруденция; Коммуникация, медиа и дизайн; Гуманитарные науки; Информационные технологии и другие.",
    "Студенты могут поступать в НИУ ВШЭ после прохождения вступительных экзаменов или конкурса. Также есть возможность поступить на основе результатов ОГЭ или ЕГЭ. Подробнее можно узнать на официальном сайте вуза.",
    "Для поступления в НИУ ВШЭ необходимо подать заявление на сайте университета, предоставить документы о среднем образовании (диплом и аттестат), а также документы об английском языке (TOEFL или IELTS). Подробнее можно узнать на официальном сайте университета.",
    "Да, существуют вступительные экзамены и тесты для поступления в НИУ ВШЭ. Для получения подробной информации рекомендуется посетить официальный сайт университета или обратиться в приемную комиссию. При необходимости можно также обратиться к консультантам или преподавателям для помощи в подготовке.",
    "Да, предоставляются стипендии и гранты для студентов НИУ ВШЭ. Информацию о них можно найти на официальном сайте университета. Также рекомендуется обратиться в службу поддержки студентов.",
    "Образовательный онлайн-проект 'Карьерные перспективы' предоставляет выпускникам НИУ ВШЭ с инвалидностью доступ к мастер-классам и тренингам, помогая им найти работу и профессионально развиваться. Проект реализуется при поддержке различных организаций."
]


In [14]:
data_mistral = [
    "НИУ ВШЭ предлагает различные факультеты и программы, включая экономику, менеджмент, компьютерные науки, юриспруденцию, коммуникацию, медиа и дизайн, гуманитарные науки и другие.",
    "Требования для поступления в НИУ ВШЭ зависят от выбранной программы и уровня образования. В целом, требуется заполнить заявку, сдать экзамены и представить документы.",
    "Для поступления в НИУ ВШЭ необходимо подать заявку, сдать экзамены и предоставить необходимые документы, в том числе свидетельство о завершении среднего образования, справку о нетрудоспособности (если студент), справку о нетрудоспособности (если ас",
    "Да, есть вступительные экзамены и тесты для поступления в НИУ ВШЭ. Если вы хотите поступить на магистерскую программу \"Экономический анализ\", вам потребуется сдать тесты по математике и экономике.",
    "Стипендии или гранты для студентов НИУ ВШЭ предоставляются в зависимости от конкретной программы и условий.",
    "Карьерные возможности для выпускников НИУ ВШЭ очень широки и разнообразны. Они могут работать в различных сферах, таких как наука, образование, бизнес, государственное управление и многих других."
]


In [12]:
import torch
import torch.nn.functional as F

def get_cos_sim(sentence1, sentence2, embeddings):
    emb1 = embeddings(sentence1)
    emb2 = embeddings(sentence2)

    if emb1.dim() > 1:
        emb1 = emb1[0]
    if emb2.dim() > 1:
        emb2 = emb2[0]

    cos_sim = F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0), dim=1)
    return cos_sim.item()


In [20]:
for i in range(len(data_mistral)):
  question = questions_hse[i]
  sentence1 = data_mistral[i]
  sentence2 = data_llama[i]
  similarity = get_cos_sim(sentence1, sentence2, get_embedding)
  print("Cosine similarity between the two sentences:", similarity)
  similarity = get_cos_sim(question, sentence2, get_embedding)
  print("Cosine similarity for llama:", similarity)
  similarity = get_cos_sim(question, sentence1, get_embedding)
  print("Cosine similarity for mistral:", similarity)
  print()

Cosine similarity between the two sentences: 0.8757205009460449
Cosine similarity for llama: 0.6744743585586548
Cosine similarity for mistral: 0.7873799800872803

Cosine similarity between the two sentences: 0.5783901810646057
Cosine similarity for llama: 0.6704692244529724
Cosine similarity for mistral: 0.8202326893806458

Cosine similarity between the two sentences: 0.8118595480918884
Cosine similarity for llama: 0.6119443774223328
Cosine similarity for mistral: 0.5856632590293884

Cosine similarity between the two sentences: 0.6131017804145813
Cosine similarity for llama: 0.5034855604171753
Cosine similarity for mistral: 0.7605272531509399

Cosine similarity between the two sentences: 0.8208038210868835
Cosine similarity for llama: 0.7950535416603088
Cosine similarity for mistral: 0.8903334736824036

Cosine similarity between the two sentences: 0.7059042453765869
Cosine similarity for llama: 0.6108965873718262
Cosine similarity for mistral: 0.6715223789215088

